In [4]:
from NeuralNetwork import NeuralNetwork, train_nn, performance, split
from _functions import *
import numpy as np
from time import time

start = time()

# Get a list of all the client's models
client_models = get_dirlist("models")
client_models.remove("server")

# Find the "servers" weights
# This, by default is the first trained model that a client spits out
serverWIH= np.load("models/" + client_models[0] +"/wih.npy")
serverWHO = np.load("models/" + client_models[0]+ "/who.npy")

def avg_func(w, delta, percentage):
    return w + delta * 1/(100/percentage)

def gentle_avg(m1, m2, split_percentage):
    combined = m1
    for r_index, row in enumerate(m1):
        for w_index, weight in enumerate(row):
            clientW = m2[r_index][w_index]
            serverW = weight
            delta = (clientW - serverW)
            combined[r_index, w_index] = avg_func(serverW, delta, split_percentage)
    return combined

totalAcc = 0

# Combine/Merge all the models
for model in client_models:
    
    clientWIH = np.load("models/" + model +"/wih.npy")
    clientWHO = np.load("models/" + model +"/who.npy")
    
    #serverWIH = np.mean( np.array([ serverWIH, clientWIH ]), axis=0 )
    #serverWHO = np.mean( np.array([ serverWHO, clientWHO ]), axis=0 )
    serverWIH = gentle_avg(serverWIH, clientWIH, 20)
    serverWHO = gentle_avg(serverWHO, clientWHO, 20)
    
    # number of input, hidden and output nodes
    input_nodes = 784
    hidden_nodes = 200
    output_nodes = 10
    learning_rate = 0.1

    # Create the neural network object
    n = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)
    n.update_info(serverWIH, serverWHO)
    
    perf = performance(n)
    totalAcc += perf
    print("Server performance after merge with " + model +": " + str(perf))

print("Final server performance: " + str(round(totalAcc / len(client_models), ndigits=3)))
print("")
print("Merging took: " + str(round(time() - start, ndigits = 3)) + " seconds")

Server performance after merge with mnist1526047304.1800025: 0.9715
Server performance after merge with mnist1526053021.527943: 0.9705
Final server performance: 0.971

Merging took: 6.194 seconds
